In [ ]:
from __future__ import annotations
import polars as pl
import seaborn as sns
import tqdm
from databento_dbn import FIXED_PRICE_SCALE, UNDEF_PRICE
# plot price of each over time
import plotly.graph_objs as go
import plotly.offline as pyo
import pandas as pd
import numpy as np
import datetime as dt  # For plotting x-axis as dates
import matplotlib.pyplot as plt

In [ ]:
data = pl.read_parquet("/Users/danny/trading/databento/mbp.parquet")
data = data.filter((pl.col("symbol") == "GOOGL") | (pl.col("symbol") == "GOOG"))
data = data.filter(pl.col("price") != UNDEF_PRICE)
data = data.with_columns(pl.col("ts_event") - pl.col("ts_event").min())
data = data.filter(pl.col("price") < 200370000000.0)
data.shape